# Upcoming Meetups

This notebook queries [meetup.com](https://meetup.com) for upcoming events matching a topic and having at least 3 RSVPs. It then gives a summary of those meetups in tabular and map form.

 To run this notebook:

0. `pip install folium pandas numpy requests` if you haven't already.
1. Set your API key value
2. Set your search topic.
3. Run All.

In [ ]:
API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
TOPIC = 'bluemix'

In [ ]:
import requests
import numpy as np
import pandas as pd
import folium
from IPython.display import HTML

In [ ]:
pd.set_option('display.max_colwidth', 500)

Request the open events from the Meetup.com API.

In [ ]:
r = requests.get("https://api.meetup.com/2/open_events", params={'topic': TOPIC, 'key': API_KEY})

In [ ]:
r.raise_for_status()

In [ ]:
df = pd.DataFrame(r.json()['results'])

Convert the times since epoch in $\mu s$ to datetime objects, accounting for timezone offset. Hereafter, the times will be local to the meetup venue.

In [ ]:
df['localtime'] = pd.to_datetime(df.time+df.utc_offset, unit='ms')

Create a human readable description of the location down to the city level, if possible.

In [ ]:
def text_location(venue):
    '''
    Return city, state, country, omitting any piece that isn't available.
    '''
    loc = []
    if pd.isnull(venue): return ''
    if 'city' in venue:
        loc.append(venue['city'])
    if 'state' in venue:
        loc.append(venue['state'])
    if 'country' in venue:
        loc.append(venue['country'].upper())
    return ', '.join(loc)

In [ ]:
df['location'] = df.venue.apply(text_location)

Turn the event name into a link to its page on meetup.com.

In [ ]:
df['link_name'] = df.apply(lambda row: '<a href="{row[event_url]}" target="_blank">{row[name]}</a>'.format(row=row), axis=1)

Use the HTML output feature instead of static markup so that the topic name appears.

In [ ]:
HTML('<h2>Table of Upcoming <em>{}</em> Meetups</h2>'.format(TOPIC))

In [ ]:
HTML(df[['link_name', 'localtime', 'location', 'yes_rsvp_count']].to_html(escape=False))

In [ ]:
HTML('<h2>Map of Upcoming <em>{}</em> Meetups</h2>'.format(TOPIC))

In [ ]:
def map_marker(row):
    '''
    Returns a dictionary with the lat/long location of an event venue as well
    as a popup containing a link to its meetup.com page. Filters events
    with no valid lat/long location.
    '''
    if pd.isnull(row['venue']): return None
    lat = row['venue'].get('lat', 0.)
    lon = row['venue'].get('lon', 0.)
    if lat == 0 or lon == 0: return None
    return dict(
        location=[lat, lon],
        popup=row['link_name']
    )

In [ ]:
m = folium.Map(location=[45, -40], zoom_start=2)

In [ ]:
for i, row in df.iterrows():
    marker = map_marker(row)
    if marker:
        m.simple_marker(**marker)

In [ ]:
m._build_map()

Idea: We might want show the venues of RSVPs in realtime on a map along with the locations of our meetups.

In [ ]:
HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=m.HTML.replace('"', '&quot;')))